In [ ]:
import json
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
import joblib

In [ ]:
!pip install skl2onnx onnx onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
with open('text_intents_dataset.json', 'r') as f:
    data = json.load(f)

texts, labels = zip(*data)

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
label_names = list(label_encoder.classes_)
print("Classes:", label_names)

Classes: [np.str_('cycle_font_next'), np.str_('cycle_font_prev'), np.str_('decrease_contrast'), np.str_('decrease_font'), np.str_('decrease_letter_spacing'), np.str_('decrease_line_spacing'), np.str_('decrease_word_spacing'), np.str_('increase_contrast'), np.str_('increase_font'), np.str_('increase_letter_spacing'), np.str_('increase_line_spacing'), np.str_('increase_word_spacing'), np.str_('other'), np.str_('undo_changes')]


In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(
        lowercase=True,
        max_features=300,    # keep small for ONNX size
        stop_words='english'
    )),
    ('clf', LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        max_iter=300,
        C=3.0,
        random_state=42
    )),
])

pipeline.fit(texts, y)
print("Train accuracy:", pipeline.score(texts, y))

Train accuracy: 0.9593023255813954


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
onnx_model = convert_sklearn(
    pipeline,
    initial_types=[('input', StringTensorType([None, 1]))],
    options={id(pipeline): {'zipmap': False}}
)
with open("intent-classifier.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())
print("Saved intent-classifier.onnx")

Saved intent-classifier.onnx


In [ ]:
with open("intent_labels.json", "w") as f:
    json.dump(label_names, f)

vocab = pipeline.named_steps['vect'].vocabulary_

In [ ]:
inv_vocab = {idx: word for word, idx in vocab.items()}
sorted_vocab = [inv_vocab[i] for i in range(len(inv_vocab))]
with open("intent_vectorizer_vocab.json", "w") as f:
    json.dump(sorted_vocab, f)

print("Saved intent_labels.json and intent_vectorizer_vocab.json")

Saved intent_labels.json and intent_vectorizer_vocab.json


In [ ]:
joblib.dump(pipeline, "intent_pipeline.joblib")
print("Done!")

Done!


In [ ]:
pipeline = joblib.load("intent_pipeline.joblib")

In [ ]:
vectorizer = pipeline.named_steps["vect"]          # CountVectorizer
clf = pipeline.named_steps["clf"]                  # LogisticRegression
label_encoder = None                               # We used LabelEncoder earlier

In [ ]:
with open("intent_labels.json", "r") as f:
    label_names = json.load(f)

vocab = vectorizer.vocabulary_

In [ ]:
inv_vocab = {idx: tok for tok, idx in vocab.items()}
ordered_vocab = [inv_vocab[i] for i in range(len(inv_vocab))]

weights = {
    "labels": label_names,
    "vocab": ordered_vocab,
    "coef": clf.coef_.tolist(),        # shape: [num_classes, vocab_size]
    "intercept": clf.intercept_.tolist()
}

with open("intent_logreg_weights.json", "w") as f:
    json.dump(weights, f)

print("Wrote intent_logreg_weights.json")

Wrote intent_logreg_weights.json
